In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import col, countDistinct, isnan, when, count, round, substring_index,substring, split, regexp_replace, udf
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType, IntegerType
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, LinearSVC

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tabulate import tabulate






 ## Since we write local [*] in the master, it will use all cores in our machine. If we said local [4] it will work with 4 cores.



 ## getOrCreate is used to create a SparkSession if not present.

In [2]:

spark=SparkSession.builder\
    .master("local[*]")\
    .appName("LoanApproval")\
    .getOrCreate()


In [3]:

sc=spark.sparkContext


 ## Read Data - SBAnational.csv

In [4]:

data_path="../data/SBAnational.csv"


In [5]:

loan_df =  spark.read.csv(data_path, header=True, inferSchema=True, quote='"', escape='"', multiLine=True)


In [6]:

loan_df.show(5)
print('=====================')
print("Number of rows in the dataframe:")
loan_df_count = loan_df.count()
print(loan_df_count)
print('=====================')
print("Schema of the dataframe:")
loan_df.printSchema() #prints the dataframe schema
print('=====================')
print("Columns in the dataframe:")
print(loan_df.columns) 


+-------------+--------------------+------------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|LoanNr_ChkDgt|                Name|        City|State|  Zip|                Bank|BankState| NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|    SBA_Appv|
+-------------+--------------------+------------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|   1000014003|      ABC HOBBYCRAFT|  EVANSVILLE|

 # Preprocessing and cleaning

 ### Report

In [7]:

# =========================================================================
# =========================================================================
# ============================= DF REPORT =================================
# =========================================================================
# =========================================================================
def report_df(df, header):
    # Calculate the total number of rows
    rdd_count = df.count()

    # Initialize lists to store column statistics
    col_names = []
    data_types = []
    unique_samples = []
    num_uniques = []
    nan_percentages = []
    report_data = []

    # Iterate over each column
    for col_name in header:
        print(col_name)
        # Append column name
        col_names.append(col_name)
        selected_col = col(col_name)
        selected_col_df = df.select(selected_col)

        # Determine data type
        dtype = selected_col_df.dtypes[0][1]
        data_types.append(dtype)
        distinct_df = selected_col_df.distinct()
        # Collect unique values
        unique_sample = [row[col_name] for row in distinct_df.limit(2).collect()]
        unique_samples.append(unique_sample)

        # Count number of unique values
        n_unique = distinct_df.count()
        num_uniques.append(n_unique)

        # Calculate percentage of NaN values
        none_percentage_val = df.filter(selected_col.isNull()).count() / rdd_count * 100
        nan_percentages.append(none_percentage_val)
        report_data.append([col_name, dtype, unique_sample, n_unique, none_percentage_val])

    return report_data


In [8]:

report_res = report_df(loan_df, loan_df.columns)



LoanNr_ChkDgt
Name
City
State
Zip
Bank
BankState
NAICS
ApprovalDate
ApprovalFY
Term
NoEmp
NewExist
CreateJob
RetainedJob
FranchiseCode
UrbanRural
RevLineCr
LowDoc
ChgOffDate


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\bemoi\miniconda3\envs\v38_env\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\bemoi\miniconda3\envs\v38_env\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\Users\bemoi\miniconda3\envs\v38_env\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:


# Display the result
column_names = ['Column', 'Type', 'Unique Sample', 'N Unique', '%None']
print(tabulate(report_res, headers=column_names, tablefmt='grid'))


In [ ]:

def show_percentage_of_each_value_in_column(df, df_count,col_name):
    # Calculate percentage of 0s and 1s
    percentage_df = df.groupBy(col_name).agg((count("*") / df_count).alias("Percentage"))

    # Round percentage values to two decimal places
    percentage_df = percentage_df.withColumn("Percentage", round(col("Percentage") * 100, 2))

    # sort the dataframe by percentage descending
    percentage_df = percentage_df.sort(col("Percentage").desc())

    # Show result
    percentage_df.show()


 ### 1. LoanNr_ChkDgt - ID

 Drop the column as it is an ID column and does not provide any information for the analysis.

In [ ]:

loan_df = loan_df.drop('LoanNr_ChkDgt')


 ### 2. Name - Name of Borrower

 Drop the column as it is a name column and does not provide any information for the analysis.

In [ ]:

col_name = 'Name'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)



In [ ]:

# percentage of unique values in the city
city_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in City: {city_unique_count}")
city_percentage = city_unique_count / loan_df_count * 100
print(f"Percentage of unique values in City: {city_percentage:.2f}%")


 Drop as most of the names are unique

In [ ]:

loan_df = loan_df.drop('Name')


 ### 3. City - City of Borrower



In [ ]:

# Count the occurrences of each value in city column
col_name = 'City'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

# percentage of unique values in the city
city_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in City: {city_unique_count}")
city_percentage = city_unique_count / loan_df_count * 100
print(f"Percentage of unique values in City: {city_percentage:.2f}%")


In [ ]:

unique_city_df = loan_df.select(col_name).groupBy(col_name).agg((count("*")).alias("Count")).sort(col("Count").desc())
unique_city_df.show()


In [ ]:

# delete null values
loan_df = loan_df.dropna(subset=[col_name])



 ### 4. State - State of Borrower

In [ ]:

col_name = 'State'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

# percentage of unique values in the city
state_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in State: {state_unique_count}")
state_percentage = state_unique_count / loan_df_count * 100
print(f"Percentage of unique values in State: {state_percentage:.2f}%")


In [ ]:

# delete nulls
loan_df = loan_df.dropna(subset=[col_name])



 ### 5. Zip - Zip code of Borrower

In [ ]:

col_name = 'Zip'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

# percentage of unique values in the city
zip_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in Zip: {zip_unique_count}")
zip_percentage = zip_unique_count / loan_df_count * 100
print(f"Percentage of unique values in Zip: {zip_percentage:.2f}%")


In [ ]:

unique_zip_df = loan_df.select(col_name).groupBy(col_name).agg((count("*")).alias("Count")).sort(col("Count").desc())
unique_zip_df.show()


In [ ]:

loan_df = loan_df.drop("col_name")


 ### 6. Bank - Name of the bank that gave the loan

In [ ]:

col_name = 'Bank'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)
loan_df = loan_df.dropna(subset=[col_name])

# percentage of unique values in the city
bank_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in Zip: {bank_unique_count}")
bank_percentage = bank_unique_count / loan_df_count * 100
print(f"Percentage of unique values in Zip: {bank_percentage:.2f}%")


 ### 7. BankState - State of Bank

In [ ]:

col_name = 'BankState'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)
loan_df = loan_df.dropna(subset=[col_name])

# percentage of unique values in the city
bankState_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in Zip: {bankState_unique_count}")
bankState_percentage = bankState_unique_count / loan_df_count * 100
print(f"Percentage of unique values in Zip: {bankState_percentage:.2f}%")


 ### 8. NAICS - North American Industry Classification System code for the industry where the business is located

In [ ]:

col_name='NAICS'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

# percentage of unique values in the city
zip_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in Zip: {zip_unique_count}")
zip_percentage = zip_unique_count / loan_df_count * 100
print(f"Percentage of unique values in Zip: {zip_percentage:.2f}%")


In [ ]:

# # Convert NAICS code into related sector

# Extract first two characters of NAICS code
first_two_chars = substring(loan_df["NAICS"], 1, 2)
# print(first_two_chars)[0]

# Apply mapping using when and otherwise
loan_df = loan_df.withColumn("Sector",
    first_two_chars
)
loan_df = loan_df.drop("NAICS")
col_name='Sector'
# # Function to get the sector for a given NAICS code
# def get_sector(naics_code):
#     return naics_to_sector[str(naics_code)]

# # Register UDF
# get_sector_udf = udf(get_sector, StringType())

# # Apply UDF to create a new column 'Sector'
# loan_df = loan_df.withColumn("Sector", get_sector_udf(loan_df["Sector"]))
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

naics_to_sector = {
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31-33': 'Manufacturing',
    '42': 'Wholesale Trade',
    '44-45': 'Retail Trade',
    '48-49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
}
col_name='Sector'

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == 32, 31)
                   .when(col(col_name) == 33, 31)
                   .when(col(col_name) == 45, 44)
                   .when(col(col_name) == 49, 48))

# loan_df = loan_df.withColumn(col_name, 
#                    when(col(col_name) == "Y", 1)
#                    .otherwise(0)
#                    .cast("int"))


In [ ]:

loan_df.show(5)


 ### 9. ApprovalDate - Date SBA commitment issued

In [ ]:

col_name = 'ApprovalDate'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

# the full date has too much detail, so we will extract the month only
# first, remove the nulls
col_name = 'ApprovalDate'
print(f"Number of rows before removing nulls: {loan_df_count}")
loan_df = loan_df.filter(loan_df[col_name].isNotNull())
loan_df_count = loan_df.count()
print(f"Number of rows after removing nulls: {loan_df_count}")
# split on '-', get the second element
loan_df = loan_df.withColumn("ApprovalMonth", split(col(col_name), "-")[1])
loan_df = loan_df.drop(col_name)
col_name = 'ApprovalMonth'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,"ApprovalMonth")


 ### 10. ApprovalFY - Fiscal Year of commitment

 Drop the column as it is a date column and does not provide any information for the analysis.

In [ ]:

loan_df = loan_df.drop('ApprovalFY')


 ### 11. Term - Loan term in months

In [ ]:

col_name = 'Term'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

# percentage of unique values in the city
term_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in Zip: {term_unique_count}")
term_percentage = term_unique_count / loan_df_count * 100
print(f"Percentage of unique values in Zip: {term_percentage:.2f}%")


In [ ]:

loan_df = loan_df.withColumn("Term_category", 
                             when((col(col_name) <=90),'Below 3 months')
                             .when(((col(col_name)>90) & (col(col_name)<=180)), '3-6 months')
                             .when(((col(col_name)>180) & (col(col_name)<=365)),  '6-12 months')
                             .otherwise('More Than a Year'))
loan_df = loan_df.drop(col_name)


In [ ]:

col_name = "Term_category"
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 ### 12. NoEmp - Number of Business Employees

In [ ]:

col_name = 'NoEmp'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

# percentage of unique values in the city
zip_unique_count = loan_df.select(col_name).distinct().count()
print(f"Number of unique values in Zip: {zip_unique_count}")
zip_percentage = zip_unique_count / loan_df_count * 100
print(f"Percentage of unique values in Zip: {zip_percentage:.2f}%")


 ### 13. NewExist - 1 = Existing business, 2 = New business

In [ ]:

col_name = 'NewExist'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 Drop rows with 0 or Null

In [ ]:

col_name = 'NewExist'
print(f"Number of rows before removing 0s and nulls: {loan_df_count}")
loan_df = loan_df.filter(loan_df[col_name] != 0)
loan_df = loan_df.filter(loan_df[col_name].isNotNull())
loan_df_count = loan_df.count()
print(f"Number of rows after removing 0s and nulls: {loan_df_count}")


In [ ]:

show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == "2", 1)
                   .otherwise(0)
                   .cast("int"))


In [ ]:

show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 ### 14. CreateJob - Number of jobs created

In [ ]:

col_name='CreateJob'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

# loan_df = loan_df.drop(col_name)


 ### 15. RetainedJob - Number of jobs retained

In [ ]:

col_name='RetainedJob'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

loan_df = loan_df.drop(col_name)


 ### 16. FranchiseCode - Franchise code, (00000 or 00001) = No franchise

In [ ]:

col_name='FranchiseCode'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 We don't care about the franchise code, we only care if there is a franchise or not

In [ ]:

# make 0 or 1 = 0, anything else = 1
loan_df = loan_df.withColumn("IsFranchise", when((col(col_name) == 0) | (col(col_name) == 1), 0).otherwise(1))


In [ ]:

col_name = 'IsFranchise'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

loan_df = loan_df.drop('FranchiseCode')


 ### 17. UrbanRural - 1 = Urban, 2 = rural, 0 = undefined

In [ ]:

col_name = 'UrbanRural'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 ### 18. RevLineCr - Revolving line of credit: Y = Yes, N = No

In [ ]:

col_name = 'RevLineCr'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 Filter only N and Y

In [ ]:

col_name = 'RevLineCr'
print(f"Number of rows before filtering: {loan_df_count}")
loan_df = loan_df.filter(loan_df[col_name].isin('N', 'Y'))
loan_df_count = loan_df.count()
print(f"Number of rows after filtering: {loan_df_count}")
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 Transform N and Y to 0 and 1

In [ ]:

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == "Y", 1)
                   .otherwise(0)
                   .cast("int"))


 ### 19. LowDoc - LowDoc Loan Program: Y = Yes, N = No

In [ ]:

col_name = "LowDoc"
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 Filter only N and Y

In [ ]:

col_name = 'LowDoc'
print(f"Number of rows before filtering: {loan_df_count}")
loan_df = loan_df.filter(loan_df[col_name].isin('N', 'Y'))
loan_df_count = loan_df.count()
print(f"Number of rows after filtering: {loan_df_count}")
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 Transform N and Y to 0 and 1

In [ ]:

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == "Y", 1)
                   .otherwise(0)
                   .cast("int"))


 ### 20. ChgOffDate - The date when a loan is declared to be in default

 Drop the column due to the high number of missing values.

In [ ]:

loan_df = loan_df.drop('ChgOffDate')


 ### 21. DisbursementDate - Date when loan was disbursed

In [ ]:

loan_df = loan_df.drop('DisbursementDate')


 ### 22. DisbursementGross - Amount disbursed

In [ ]:

loan_df = loan_df.drop('DisbursementGross')


 ### 23. BalanceGross - Gross amount outstanding

In [ ]:

col_name = 'BalanceGross'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 Drop as most of the values are 0

In [ ]:

loan_df = loan_df.drop('BalanceGross')


 ### 24. MIS_Status - Target variable

 Delete rows that have null target value (MIS_Status)

In [ ]:

col_name ="MIS_Status"
show_percentage_of_each_value_in_column(loan_df, loan_df_count, col_name)


In [ ]:

# drop rows with null values in MIS_Status column
loan_df = loan_df.dropna(subset=[col_name])
show_percentage_of_each_value_in_column(loan_df, loan_df_count, col_name)



 ### Replace target values with 0 and 1

 Target value column is: MIS_Status

 "P I F" = 1

 "CHGOFF" = 0

In [ ]:

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == "P I F", 1)
                   .otherwise(0)
                   .cast("int"))


 Show the percentage of:

 - Paid in full loans (approved loans), MIS_Status = 1

 - Charged off loans (rejected loans), MIS_Status = 0

In [ ]:

show_percentage_of_each_value_in_column(loan_df, loan_df_count, col_name)


 ### 25. ChgOffPrinGr - Charged-off amount

In [ ]:

col_name = 'ChgOffPrinGr'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 Drop this column as it will leak info to the column, because if the value is 0, this means that the loan is charged off

In [ ]:

loan_df = loan_df.drop('ChgOffPrinGr')


 ### 26. GrAppv - Gross amount of loan approved by bank

In [ ]:

col_name = "GrAppv"
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


 #### Clean this column

 - Remove $

 - Remove ,

 - Convert to float

In [ ]:

loan_df = loan_df.withColumn("clean_GrAppv", regexp_replace("GrAppv", "\$", ""))  # Remove $
loan_df = loan_df.withColumn("clean_GrAppv", regexp_replace("clean_GrAppv", ",", ""))  # Remove comma
loan_df = loan_df.withColumn("clean_GrAppv", col("clean_GrAppv").cast("float"))
col_name = "clean_GrAppv"
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


In [ ]:

loan_df = loan_df.drop('GrAppv')


 ### 27. SBA_Appv - SBA's guaranteed amount of approved loan

 Drop as we don't know this amount in the future

In [ ]:

loan_df = loan_df.drop('SBA_Appv')


 ### Final schema

In [ ]:

loan_df.printSchema()


 ### Check duplicated rows based on all columns



In [ ]:

print("Number of duplicate rows in the dataframe:")
loan_df_duplicates = loan_df_count - loan_df.dropDuplicates().count()
print(loan_df_duplicates)


 ### Final DF Count

In [ ]:

loan_df_count = loan_df.count()
print(f"Final DF count: {loan_df_count}")


In [ ]:

report_res = report_df(loan_df, loan_df.columns)
# Display the result
column_names = ['Column', 'Type', 'Unique Sample', 'N Unique', '%None']
print(tabulate(report_res, headers=column_names, tablefmt='grid'))